In [1]:
import mongoengine
from mongoengine import *
from mongoengine.queryset.visitor import Q
from dbmodels import *
from hashfunction import *
from readref import *

import pandas as pd
from bson.objectid import ObjectId
import configparser
from multiprocessing import Pool
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

In [182]:
# Read the confidentials.
credentials = configparser.ConfigParser()
credentials.read('credentials.ini')

# Connect to the database.
connect(
    db = credentials.get('lb', 'db'),
    username = credentials.get('lb', 'username'),
    password = credentials.get('lb', 'password'),
    host = credentials.get('lb', 'host'), 
    port = int(credentials.get('lb', 'port')), 
)

MongoClient(host=['128.178.60.49:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

# Get Filtered References

Assume the full reference should be like: 
- Author + Title + xxx
- Title + Author + xxx
- xxx + Author + Title
- xxx + Title + Author

In [35]:
refs = Reference.objects.filter(Q(contents__1__tag='author', contents__2__tag='title') |
                                Q(contents__2__tag='author', contents__1__tag='title') |
                                Q(contents__2__tag='author', contents__3__tag='title') |
                                Q(contents__3__tag='author', contents__2__tag='title'))

In [36]:
Reference.objects.count()

3850581

In [37]:
refNum = refs.count()
refNum

1227978

In [7]:
# test
for r in refs.limit(3):
    print(read_ref(r))
    print("\n")

{'doc_id': ObjectId('595f9e11fe7683316b2dd3f5'), 'ref_id': ObjectId('58e4ce00fe7683152b55e325'), 'surface': 'ATTI DELL’ATENEO necrologio GIOVANNI MANETTI Fra i soci scomparsi, Γ Ateneo deve rimpiangere il Comm. Dr. Giovanni Manetti, cara e degnissima figura di vero galantuomo. A riposo, dopo avere a lungo coperto a Venezia Γ Ufficio di Consigliere di Prefettura, fu, durante la grande guerra, Commissario per i profughi Veneti rico- verati lungo i lidi Anconitani, profondendo in quest’ opera i tesori della sua filantropia illuminata e squisita. Fu poi Commissario per la fondazione Querini Stampalia e attivissimo Vice- presidente dell’ opera « Nave Asilo Scilla per gli orfani di pescatori » ; presiedette infine per lunghi anni, con costante e fervido amore, la « Cassa del piccolo Credito » e la «S. A. Cooperativa Bagni di Mare fra impiegati e professionisti» al Lido.', 'order': 68001, 'author': ['ATTI DELL’ATENEO necrologio GIOVANNI MANETTI'], 'title': ['Fra i soci scomparsi, Γ Ateneo dev

/Users/suntao/anaconda3/envs/lb/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: generator 'QuerySet._iter_results' raised StopIteration
  from ipykernel import kernelapp as app


# Apply Hash Functions

In [8]:
def read_hash(ref):
    '''
    Read reference and generate hashes.
    '''
    citation = read_ref(ref)
    d = {
        'doc_id' : citation['doc_id'],
        'ref_id' : citation['ref_id'],
        'surface' : citation['surface'],
        'hashes' : hashCombined(citation)
    }
    return d

In [12]:
pool = Pool()

/Users/suntao/anaconda3/envs/lb/lib/python3.6/site-packages/pymongo/topology.py:145: UserWarning: MongoClient opened before fork. Create MongoClient with connect=False, or create client after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#pymongo-fork-safe>
  "MongoClient opened before fork. Create MongoClient "
/Users/suntao/anaconda3/envs/lb/lib/python3.6/site-packages/pymongo/topology.py:145: UserWarning: MongoClient opened before fork. Create MongoClient with connect=False, or create client after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#pymongo-fork-safe>
  "MongoClient opened before fork. Create MongoClient "
/Users/suntao/anaconda3/envs/lb/lib/python3.6/site-packages/pymongo/topology.py:145: UserWarning: MongoClient opened before fork. Create MongoClient with connect=False, or create client after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/

  File "/Users/suntao/anaconda3/envs/lb/lib/python3.6/site-packages/pymongo/cursor.py", line 1036, in _refresh
    self.__collation))
  File "/Users/suntao/anaconda3/envs/lb/lib/python3.6/site-packages/pymongo/mongo_client.py", line 905, in _send_message_with_response
    exhaust)
  File "/Users/suntao/anaconda3/envs/lb/lib/python3.6/site-packages/pymongo/cursor.py", line 1036, in _refresh
    self.__collation))
  File "/Users/suntao/anaconda3/envs/lb/lib/python3.6/site-packages/pymongo/cursor.py", line 873, in __send_message
    **kwargs)
  File "/Users/suntao/anaconda3/envs/lb/lib/python3.6/site-packages/pymongo/cursor.py", line 873, in __send_message
    **kwargs)
  File "/Users/suntao/anaconda3/envs/lb/lib/python3.6/site-packages/pymongo/mongo_client.py", line 916, in _reset_on_error
    return func(*args, **kwargs)
  File "/Users/suntao/anaconda3/envs/lb/lib/python3.6/site-packages/pymongo/mongo_client.py", line 905, in _send_message_with_response
    exhaust)
  File "/Users/sunta

In [10]:
# test
temp = pool.map(read_hash, refs.limit(3))
df = pd.DataFrame(temp)
df.head()

,doc_id,hashes,ref_id,surface
0,595f9e11fe7683316b2dd3f5,"[giovanni#asilo#scilla, ateneo#soci#scomparsi,...",58e4ce00fe7683152b55e325,ATTI DELL’ATENEO necrologio GIOVANNI MANETTI F...
1,595f9e37fe7683316b2dd4b4,"[hans#drawings#the, hans#1943#221, the#drawing...",58e4ce00fe7683152b55e328,Hans Tietzte - Tietzte Conrad The drawings of ...
2,595f9e37fe7683316b2dd4b4,"[devant#als, heil#palma#devant#als#zeichner#ja...",58e4ce00fe7683152b55e32b,"Heil Palma devant als Zeichner, in ""in Jahrbuc..."


In [ ]:
df = pd.read_pickle('hashsample.pickle')

In [13]:
df = pd.DataFrame()
for i in range(0, refNum, 50000):
    print("Now we are at {}.".format(i))
    temp = pool.map(read_hash, refs[i:i+50000])
    df_temp = pd.DataFrame(temp)
    df = df.append(df_temp)

Now we are at 0.
Now we are at 50000.
Now we are at 100000.
Now we are at 150000.
Now we are at 200000.
Now we are at 250000.
Now we are at 300000.
Now we are at 350000.
Now we are at 400000.


KeyboardInterrupt: 

/Users/suntao/anaconda3/envs/lb/lib/python3.6/site-packages/pymongo/topology.py:145: UserWarning: MongoClient opened before fork. Create MongoClient with connect=False, or create client after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#pymongo-fork-safe>
  "MongoClient opened before fork. Create MongoClient "
/Users/suntao/anaconda3/envs/lb/lib/python3.6/site-packages/pymongo/topology.py:145: UserWarning: MongoClient opened before fork. Create MongoClient with connect=False, or create client after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#pymongo-fork-safe>
  "MongoClient opened before fork. Create MongoClient "
/Users/suntao/anaconda3/envs/lb/lib/python3.6/site-packages/pymongo/topology.py:145: UserWarning: MongoClient opened before fork. Create MongoClient with connect=False, or create client after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/

In [14]:
df.head()

,doc_id,hashes,ref_id,surface
0,595f9e11fe7683316b2dd3f5,"[giovanni#asilo#scilla, ateneo#soci#scomparsi,...",58e4ce00fe7683152b55e325,ATTI DELL’ATENEO necrologio GIOVANNI MANETTI F...
1,595f9e37fe7683316b2dd4b4,"[hans#drawings#the, hans#1943#221, the#drawing...",58e4ce00fe7683152b55e328,Hans Tietzte - Tietzte Conrad The drawings of ...
2,595f9e37fe7683316b2dd4b4,"[devant#als, heil#palma#devant#als#zeichner#ja...",58e4ce00fe7683152b55e32b,"Heil Palma devant als Zeichner, in ""in Jahrbuc..."
3,595f9e0efe7683316b2dd3eb,"[croce#modesti#innumerevoli, croce#vigilia#mor...",58e4ce00fe7683152b55e330,LA CROCE el vialetto che girava attorno allo s...
4,595f9f57fe7683316b2dda8c,"[monticolo#1906#468, monticolo#1906#467, capit...",58e4ce00fe7683152b55e336,"Monticolo, I capitolari delle arti veneziane, ..."


In [15]:
len(df)

400000

In [16]:
pool.close()

In [17]:
df[:100000].to_pickle('hashsample1.pickle')
df[100000:200000].to_pickle('hashsample2.pickle')
df[200000:300000].to_pickle('hashsample3.pickle')
df[300000:400000].to_pickle('hashsample4.pickle')

In [ ]:
df.to_pickle('hashsample.pickle')

In [ ]:
pool.join()

# Map

In [3]:
def splitHash(df):
    s = df.apply(lambda x: pd.Series(x['hashes']),axis=1).stack().reset_index(level=1, drop=True)
    s.name = 'hashtag'
    return df.drop('hashes', axis=1).join(s)

In [58]:
df = pd.read_pickle('hashsample.pickle')

In [61]:
df_sample_split = splitHash(df)

# Hash Statistics

In [5]:
list_groupby = []
for i in range(1, 5):
    print(i)
    df_temp = pd.read_pickle(f"./pickle/hash_split{i}.pickle")
    df_temp = df_temp.groupby('hashtag').count()[['ref_id']]
    df_temp.columns = ['counts']
    list_groupby.append(df_temp)
df_all = pd.concat(list_groupby)

1
2
3
4


In [20]:
df_all = df_all.groupby('hashtag').sum()

df_all = df_all.sort_values('counts', ascending=False)

df_all.to_pickle('./pickle/hash_stat.pickle')

In [53]:
df_all = pd.read_pickle('./pickle/hash_stat.pickle')

In [54]:
# In thousands
df_all['k_group'] = df_all['counts'].apply(lambda x: (x//1000))
df_all.groupby('k_group')['counts'].count()

k_group
0    22374351
1          67
2          16
3          21
4           3
6           1
Name: counts, dtype: int64

In [55]:
# In hundreds, for those below 1,000
df_all['h_group'] = df_all['counts'].apply(lambda x: (x//100))
df_all[df_all.counts<1000].groupby('h_group')['counts'].count()

h_group
0    22369404
1        3587
2         773
3         289
4         134
5          58
6          41
7          27
8          24
9          14
Name: counts, dtype: int64

In [56]:
# In tens, for those below 100
df_all['t_group'] = df_all['counts'].apply(lambda x: (x//10))
df_all[df_all.counts<100].groupby('t_group')['counts'].count()

t_group
0    22228720
1       90892
2       24089
3       10675
4        5635
5        3282
6        2410
7        1609
8        1179
9         913
Name: counts, dtype: int64

In [57]:
# In ones, for those below 10
df_all['d_group'] = df_all['counts'].apply(lambda x: (x//1))
df_all[df_all.counts<10].groupby('d_group')['counts'].count()

d_group
1    20004324
2     1401939
3      380207
4      178342
5      100915
6       63798
7       43459
8       31810
9       23926
Name: counts, dtype: int64

In [85]:
# 1000+
df_all[df_all.k_group>0].index.values

array(['repubblica#venezia', 'san#marco', 'storia#venezia',
       'atti#convegno', 'cura#1985', 'eta#moderna', 'cura#1984',
       'cura#1987', 'cura#1983', 'cura#1993', 'cura#1996', 'cura#1986',
       'cura#1997', 'cura#1981', 'cura#1995', 'cura#1988', 'cura#1989',
       'cura#1994', 'cura#1991', 'cura#1992', 'cura#1998',
       'catalogo#mostra', 'cura#1990', 'cura#1982', 'cura#1980',
       'renaissance#venice', 'storia#cultura', 'cultura#veneta',
       'cura#1979', 'moyen#age', 'cura#1999', 'repubblica#veneta',
       'cura#2000', 'storia#arte', 'cura#2001', 'cura#2002', 'cura#2003',
       'secolo#xvi', 'cura#atti#convegno', 'cura#1978', 'cura#1973',
       'cura#1977', 'cura#1974', 'cura#1972', 'cura#1968',
       'dizionario#biografico', 'storia#italia', 'cura#1967', 'cura#2004',
       'cura#1976', 'cura#1969', 'cura#1966', 'biografico#italiani',
       'xvi#secolo', 'cura#1975', 'storia#economica', 'cura#1970',
       'civilta#veneziana', 'venezia#eta', 'cura#2005', 'cura#

1. `venezia` or `venetia` is quite common.
2. `cultura` means 'culture' and `storia` means 'history'.
3. `san#marco` is one of the six sestieri of Venice.
4. 'atti del convegno' means 'conference proceedings'. `atti#convegno` is parsed as part of title.
5. 'a cura di' means 'edited by'. `cura` is included in the hashtag as part of author due to parsing error.
6. `eta#moderna` means 'modern age'; `moyen#age`, `middle#age` means 'middle age'. 
7. 'catalogo della mostra' means 'exhibition catalog'. `catalogo#mostra` is parsed as part of title.
8. `xvi#secolo`, `secoli#xvi`, `sixteenth#century`,  `xviii#secolo`  means '16th century' or '18th century'.
9. `dizionario#biografico`, `biografico#italiani` are part of 'Dizionario biografico degli Italiani', which is a biographical dictionary.
10. `palazzo#ducale` is a palace built in Venetian Gothic style.
11. `convegno#internazionale` means 'international conference'.
12. `santa#maria` always showes up as part of church name.
13. `citta#nobilissima`, `sansovino#citta#nobilissima` points to the reference 'Venetia città nobilissima et singolare' that is commonly cited.
14. `origini#caduta` points to the reference 'Storia di Venezia nella vita privata dalle origini alla caduta della Repubblica' that is commonly cited.
15. `paolo#sarpi` is an Italian historian.
16. `cozzi#repubblica#venezia` is a combination of author 'cozzi' and title bigram 'repubblica#venezia' which is the favorite topic of this author.

**Black List**:
- `cura#...`
- `san#marco`
- `secolo`, `secoli`
- `palazzo#ducale`
- `paolo#sarpi`
- `dizionario#biografico`, `biografico#italiani`

In [64]:
# 500-900+
df_all[(df_all.counts<1000) & (df_all.h_group>=5)].index.values

array(['medio#evo', 'storia#repubblica', 'cura#2008', 'memorie#storiche',
       'seconda#meta', 'studi#storici', 'maggior#consiglio',
       'seventeenth#century', 'nobilissima#singolare', 'xvii#xviii',
       'secoli#xiv', 'andrea#palladio', 'sansovino#venetia#citta',
       'vita#privata', 'storia#patria', 'documentata#venezia',
       'venezia#secolo', 'storia#diritto', 'societa#veneta',
       'luzzatto#storia#economica', 'internazionale#studi', 'fine#secolo',
       'fifteenth#century', 'fino#secolo', 'marco#venezia',
       'museo#correr', 'sansovino#nobilissima#singolare', 'chiesa#san',
       'romanin#storia#documentata', 'medioevo#eta', 'early#modem',
       'chiesa#venezia', 'arnaldi#cultura#veneta', 'ateneo#veneto',
       'archivio#stato', 'caduta#serenissima', 'early#modern',
       'stato#venezia', 'vita#opere', 'arnaldi#storia#cultura',
       'romanin#documentata#venezia', 'patriziato#veneziano', 'cura#1955',
       'aspetti#problemi', 'xvi#xviii', 'belle#arti', 'ducal

1. `medio#evo` means middle age.
2. 

In [ ]:
df_1 = pd.read_pickle("./pickle/hash_split1.pickle")

In [106]:
for r in df_1[df_1.hashtag=='medio#evo'].head(20).surface:
    print(r)

Bensa, Il contralto di assicurazione nel medio evo, Genova, 1884, p. 32 sg.;
Augusto Torre, L’oggetto delle relazioni fra Ravenna e Venezia nel Medio evo in Miscel- lanea in onore di Roberto Cessi, Roma 1958, pp. 121-141.
G. LUZZATTO, Le vicende del Porto di Venezia, dal primo Medio Evo allo scoppio della Guerra 1914-1918, in L. CANDIDA, Il Porto di Venezia, con introduzione storica di G. LUZZATTO, Venezia 1950, pp. 7-44,
G. DE VERGOTTINI, Lineamenti storici della costituzione politica dell'Istria durante il Medio Evo, Roma 1924-25
Flavio Testi, La musica italiana nel Medio Evo e nel Rinascimento, 1, Busto Arsizio, Bra- mante Editrice, 1977, pp. 395-396, n. 15.
Pietro Selvatico, Sulla architettura e sulla scultura in Venezia dal Medio Evo sino ai nostri giorni, Venezia, Paolo Ripamonti Carpano, 1847;
Camillo Borro, Architettura del Medio Evo in Italia, Milano, Hoepli, 1880, p. 308.
PIETRO Selvatico, Sull’architettura e sulla scultura in Venezia. Dal Medio Evo sino ai giorni nostri. Stu

# Map

In [92]:
for i in [1, 2, 3, 4]:
    df = pd.read_pickle(f"./pickle/hash_split{i}.pickle")
    df.set_index('ref_id', inplace=True)
    d = df.groupby('hashtag').groups
    s = pd.Series(d)
    s = s.apply(lambda x: x.values)
    s.to_pickle(f'./pickle/hash_match{i}.pickle')

In [83]:
s_1 = pd.read_pickle('./pickle/hash_match1.pickle')

In [94]:
s_1.head()

000#000           [5919e9663dde944752f56652, 5919e96d3dde944752f...
000#100                                  [58e4ce01fe7683152b592f60]
000#1650                                 [59de7a2048e232540c5d5d85]
000#1968                                 [59de7a2048e232540c5dcfb6]
000#1968#-1#29                           [59de7a2048e232540c5dcfb6]
dtype: object

# Select without black list

In [131]:
s_1 = pd.read_pickle('./pickle/hash_match1.pickle')
s_2 = pd.read_pickle('./pickle/hash_match2.pickle')
s_3 = pd.read_pickle('./pickle/hash_match3.pickle')
s_4 = pd.read_pickle('./pickle/hash_match4.pickle')

In [170]:
df_1 = pd.read_pickle("./pickle/hash_ref1.pickle")
df_2 = pd.read_pickle("./pickle/hash_ref2.pickle")
df_3 = pd.read_pickle("./pickle/hash_ref3.pickle")
df_4 = pd.read_pickle("./pickle/hash_ref4.pickle")

In [184]:
hash_black = ['000#000']
def find_match(record):
    doc_id = record.doc_id
    hashes_all = record.hashes
    hashes = [h for h in hashes_all if h not in hash_black] # remove hashes in black list
    set_result = set()
    for s in [s_1, s_2, s_3, s_4]:
        r = [r for rs in s.reindex(hashes).dropna().values for r in rs]
             #if Reference.objects(id=r).first().get_containing_publication()!=doc_id]
        r = set(r)
        set_result = set_result.union(r) 
    return list(set_result)

In [ ]:
for i, df in enumerate([df_1, df_2, df_3, df_4]):
    print(i+1)
    df['match'] = df['hashes'].apply(find_match)
    print('store')
    df.to_pickle('./pickle/hash_ref{}.pickle'.format(i+1))

In [172]:
df_1.head()

,doc_id,hashes,ref_id,surface,match
0,595f9e37fe7683316b2dd4b4,"[conrad#1943#221, tietzte#tietzte#conrad#drawi...",58e4ce00fe7683152b55e328,Hans Tietzte - Tietzte Conrad The drawings of ...,"[59df6ec59be3c75dd98c28bc, 59e23a5988316162134..."
1,595f9e37fe7683316b2dd4b4,"[heil#devant#zeichner, devant#zeichner, heil#p...",58e4ce00fe7683152b55e32b,"Heil Palma devant als Zeichner, in ""in Jahrbuc...","[59e23a53883161621340307b, 58e4ce00fe7683152b5..."
2,595f9f57fe7683316b2dda8c,"[monticolo#1904#466, monticolo#1905#468, monti...",58e4ce00fe7683152b55e336,"Monticolo, I capitolari delle arti veneziane, ...","[59e332f68831617f5edda08e, 59e333088831617f5ee..."
3,595f9e2afe7683316b2dd476,"[morassi#1942#91, morassi#1943#91, morassi#194...",58e4ce00fe7683152b55e33f,"Morassi, Novità e precisazioni sul Tiepolo in ...","[59e23a58883161621344b833, 59df53d09be3c75c2e6..."
4,595f9e39fe7683316b2dd4bc,"[sanudo#itinerario#terra, sanudo#terra#venezia...",58e4ce00fe7683152b55e34f,"M. Sañudo, Itinerario per la terra di Venezia ...","[59df53d09be3c75c2e687c85, 59e333098831617f5ee..."


In [173]:
df_sample = df_1.iloc[:3]

In [ ]:
df_sample['new_match'] = df_sample.apply(find_match, axis=1)

# Filter Out Possible "Full" References

In [3]:
# function for the reference resolutions
def retain_reference(journal_reference, min_size=3, required_fields=["author","title"]):
    """
    Determine whether the input reference should be retained, and thus resolved, or skipped.
    This allows to skip erroneously extracted references, partial ones, etc.
    :param journal_reference: the input reference (extracted from journals in LinkedBooks)
    :type journal_reference: dict
    :param min_size: the minimum number of fields `journal_reference` must have
    :type min_size: int
    :param required_fields: the fields that must be contained in `journal_reference`
    :type required_fields: list of str
    :return: bool -- True if the reference should be retained, False otherwise
    """
    fields = [field["tag"] for field in journal_reference["contents"].values()]
    if(len(fields)>=min_size):
        if(len(set(fields).intersection(set(required_fields))) >= len(required_fields)):
            return True
        else:
            return False
    else:
        return False

In [4]:
refs = Reference.objects.filter(Q(contents__1__tag='author', contents__2__tag='title') |
                                Q(contents__2__tag='author', contents__1__tag='title') |
                                Q(contents__2__tag='author', contents__3__tag='title') |
                                Q(contents__3__tag='author', contents__2__tag='title'))

In [5]:
rs = [r for r in refs if retain_reference(r, 3, required_fields=["author","title","year"])]

/Users/suntao/anaconda3/envs/lb/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: generator 'QuerySet._iter_results' raised StopIteration
  if __name__ == '__main__':


In [6]:
len(rs)

464068

In [7]:
import pickle
with open('rs.pickle', 'wb') as f:
    pickle.dump(rs, f)

In [ ]:
# find all things that are missing by my method
rs_not = [r for r in Reference.objects() if r not in refs if retain_reference(r, 2)]
len(rs_not)

In [ ]:
rs_id = [r.id for r in rs]

In [ ]:
with open('rs_id.pickle', 'wb') as f:
    pickle.dump(rs_id, f)

In [ ]:
rs_not_id = [r.id for r in rs_not]

In [ ]:
with open('rs_not_id.pickle', 'wb') as f:
    pickle.dump(rs_not_id, f)

# Select

In [26]:
rid = ObjectId("58e4ce01fe7683152b57ee3c")

In [27]:
for i in range(13):
    df_temp = pd.read_pickle("hashsample{}.pickle".format(i+1))
    if len(df_temp[df_temp.ref_id==rid]) != 0:
        hashes = df_temp[df_temp.ref_id==rid].hashes.values[0]
        doc_id = df_temp[df_temp.ref_id==rid].doc_id.values[0]
        surface = df_temp[df_temp.ref_id==rid].surface.values[0]
        break

In [28]:
hashes

['vienna#venezia',
 'eta#moderna',
 'fornasin#eta#moderna',
 'fornasin#patria#friuli',
 'studi#venezia',
 'friuli#eta',
 'fornasin#venezia#viabilita',
 'fornasin#friuli#eta',
 'venezia#viabilita',
 'fornasin#moderna#studi',
 'fornasin#studi#venezia',
 'moderna#studi',
 'fornasin#viabilita#patria',
 'fornasin#vienna#venezia',
 'patria#friuli',
 'fornasin#vienna#venezia#viabilita#patria#friuli#eta#moderna#studi#venezia#1999',
 'viabilita#patria']

In [31]:
for i in range(13):
    df_temp = pd.read_pickle("hashsplit{}.pickle".format(i+1))
    if len(df_temp[df_temp.ref_id==rid]) != 0:
        hashtag = df_temp[df_temp.ref_id==rid].hashtag.values
        print(hashtag)
    break
del df_temp

0 ['rizzi#leoni#zara#ateneo#veneto#clxxv#1988' 'rizzi#ateneo#veneto'
 'ateneo#veneto' 'leoni#zara' 'rizzi#leoni#zara' 'vienna#venezia'
 'eta#moderna' 'fornasin#eta#moderna' 'fornasin#patria#friuli'
 'studi#venezia' 'friuli#eta' 'fornasin#venezia#viabilita'
 'fornasin#friuli#eta' 'venezia#viabilita' 'fornasin#moderna#studi'
 'fornasin#studi#venezia' 'moderna#studi' 'fornasin#viabilita#patria'
 'fornasin#vienna#venezia' 'patria#friuli'
 'fornasin#vienna#venezia#viabilita#patria#friuli#eta#moderna#studi#venezia#1999'
 'viabilita#patria']


In [66]:
df_temp = pd.read_pickle("hashsplit4.pickle")
df_temp.loc[4596]

,doc_id,ref_id,surface,hashtag
4596,5960ca56fe76836a510e8378,59de7a2048e232540c5e7e12,"FIGURE AFFACCIATE AD UNA LOGGIA Affresco, m. 4...",affacciate#1742
4596,5960ca56fe76836a510e8378,59de7a2048e232540c5e7e12,"FIGURE AFFACCIATE AD UNA LOGGIA Affresco, m. 4...",loggia#documenti#fogolari
4596,5960ca56fe76836a510e8378,59de7a2048e232540c5e7e12,"FIGURE AFFACCIATE AD UNA LOGGIA Affresco, m. 4...",figure#affresco#eseguito
4596,5960ca56fe76836a510e8378,59de7a2048e232540c5e7e12,"FIGURE AFFACCIATE AD UNA LOGGIA Affresco, m. 4...",affacciate#venezia#gallerie
4596,5960ca56fe76836a510e8378,59de7a2048e232540c5e7e12,"FIGURE AFFACCIATE AD UNA LOGGIA Affresco, m. 4...",loggia#affresco#00x1
4596,5960ca56fe76836a510e8378,59de7a2048e232540c5e7e12,"FIGURE AFFACCIATE AD UNA LOGGIA Affresco, m. 4...",affacciate#chiesa#scalzi
4596,5960ca56fe76836a510e8378,59de7a2048e232540c5e7e12,"FIGURE AFFACCIATE AD UNA LOGGIA Affresco, m. 4...",affacciate#eseguito#tiepolo
4596,5960ca56fe76836a510e8378,59de7a2048e232540c5e7e12,"FIGURE AFFACCIATE AD UNA LOGGIA Affresco, m. 4...",figure#casa#loreto
4596,5960ca56fe76836a510e8378,59de7a2048e232540c5e7e12,"FIGURE AFFACCIATE AD UNA LOGGIA Affresco, m. 4...",loggia#trasporto#santa
4596,5960ca56fe76836a510e8378,59de7a2048e232540c5e7e12,"FIGURE AFFACCIATE AD UNA LOGGIA Affresco, m. 4...",loggia#00x1#staccato


In [54]:
for i in range(1,5):
    df_temp_s = pd.read_pickle(f"hashsample{i}.pickle")
    df_temp_s = df_temp_s.set_index('ref_id').reset_index()
    df_temp_s.to_pickle(f"hashsample{i}.pickle")

In [55]:
df_temp_s = pd.read_pickle(f"hashsample1.pickle")

In [56]:
df_temp_s

,ref_id,doc_id,hashes,surface
0,58e4ce00fe7683152b55e325,595f9e11fe7683316b2dd3f5,"[giovanni#asilo#scilla, ateneo#soci#scomparsi,...",ATTI DELL’ATENEO necrologio GIOVANNI MANETTI F...
1,58e4ce00fe7683152b55e328,595f9e37fe7683316b2dd4b4,"[hans#drawings#the, hans#1943#221, the#drawing...",Hans Tietzte - Tietzte Conrad The drawings of ...
2,58e4ce00fe7683152b55e32b,595f9e37fe7683316b2dd4b4,"[devant#als, heil#palma#devant#als#zeichner#ja...","Heil Palma devant als Zeichner, in ""in Jahrbuc..."
3,58e4ce00fe7683152b55e330,595f9e0efe7683316b2dd3eb,"[croce#modesti#innumerevoli, croce#vigilia#mor...",LA CROCE el vialetto che girava attorno allo s...
4,58e4ce00fe7683152b55e336,595f9f57fe7683316b2dda8c,"[monticolo#1906#468, monticolo#1906#467, capit...","Monticolo, I capitolari delle arti veneziane, ..."
5,58e4ce00fe7683152b55e338,595f9e34fe7683316b2dd4ab,"[croce#pascoli, rodaun#tre, croce#articoli#cro...",B. Croce. Intanto erano giunti a Rodaun i tre ...
6,58e4ce00fe7683152b55e339,595f9e29fe7683316b2dd470,"[anno#cxlii, veneto#anno#cxlii, ateneo#anno#cx...",ATENEO VENETO Anno CXLII Vol. 135 - N.
7,58e4ce00fe7683152b55e33a,595f9e29fe7683316b2dd470,"[freemantlc#caro, australia#colombo#freemantlc...",VIAGGIO PER L’AUSTRALIA Da bordo S/S Toscana f...
8,58e4ce00fe7683152b55e33c,595f9e29fe7683316b2dd470,"[scarica#acqua, purtroppo#pom#pieri, purtroppo...","Purtroppo, dopo un quarto d’ora arrivarono i p..."
9,58e4ce00fe7683152b55e33d,595f9e2afe7683316b2dd476,"[postilla#tiepolesca#ipinse#venezia, tiepolesc...",UNA POSTILLA TIEPOLESCA ipinse a Venezia


In [4]:
df = pd.read_csv('./sample_csv/58e4ce00fe7683152b55e336.csv')
df.head()

,Unnamed: 0,seed_ref_id,seed_doc_id,seed_surface,ref_id,doc_id,surface,True,Hash
0,0,58e4ce00fe7683152b55e336,595f9f57fe7683316b2dda8c,"Monticolo, I capitolari delle arti veneziane, ...",58e4ce00fe7683152b55e336,595f9f57fe7683316b2dda8c,"Monticolo, I capitolari delle arti veneziane, ...",0,1
1,17,58e4ce00fe7683152b55e336,595f9f57fe7683316b2dda8c,"Monticolo, I capitolari delle arti veneziane, ...",58e4ce01fe7683152b57c0c4,595f9d70fe7683316b2dcf22,"G. Giomo e F. Visentini, Le grosse galere Vene...",0,1
2,34,58e4ce00fe7683152b55e336,595f9f57fe7683316b2dda8c,"Monticolo, I capitolari delle arti veneziane, ...",58e4ce00fe7683152b55f1db,595f9f9ffe7683316b2ddc03,"Prodi, Intervento cit., p. 216.",0,1
3,41,58e4ce00fe7683152b55e336,595f9f57fe7683316b2dda8c,"Monticolo, I capitolari delle arti veneziane, ...",58e4ce01fe7683152b57d1be,595f9d61fe7683316b2dccaf,"G. Monticolo, I capitolari delle Arti venezian...",0,1
4,48,58e4ce00fe7683152b55e336,595f9f57fe7683316b2dda8c,"Monticolo, I capitolari delle arti veneziane, ...",58e4ce00fe7683152b55f1f7,595f9fddfe7683316b2ddd4f,"Giacomo Nani, Della difesa di Venezia, a cura ...",0,1


In [19]:
df_temp = pd.read_pickle("hashsplit1.pickle")
df_temp.head()

,ref_id,doc_id,surface,hashtag
0,58e4ce00fe7683152b55e325,595f9e11fe7683316b2dd3f5,ATTI DELL’ATENEO necrologio GIOVANNI MANETTI F...,giovanni#asilo#scilla
0,58e4ce00fe7683152b55e325,595f9e11fe7683316b2dd3f5,ATTI DELL’ATENEO necrologio GIOVANNI MANETTI F...,ateneo#soci#scomparsi
0,58e4ce00fe7683152b55e325,595f9e11fe7683316b2dd3f5,ATTI DELL’ATENEO necrologio GIOVANNI MANETTI F...,veneti#rico
0,58e4ce00fe7683152b55e325,595f9e11fe7683316b2dd3f5,ATTI DELL’ATENEO necrologio GIOVANNI MANETTI F...,giovanni#fra#soci
0,58e4ce00fe7683152b55e325,595f9e11fe7683316b2dd3f5,ATTI DELL’ATENEO necrologio GIOVANNI MANETTI F...,giovanni#opera#tesori


In [3]:
def get_repr(did):
    a = Article.objects(_id=did)
    if len(a): 
        return repr(a[0])
    
    b = Book.objects(_id=did)
    if len(b): 
        return repr(b[0])
    
    return ''

In [4]:
def hash_match_csv(rid):
    # get hashes of the ref
    for i in range(13):
        df_temp = pd.read_pickle("./pickle/hashsample/hashsample{}.pickle".format(i+1))
        if len(df_temp[df_temp.ref_id==rid]) != 0:
            hashes = df_temp[df_temp.ref_id==rid].hashes.values[0]
            doc_id = df_temp[df_temp.ref_id==rid].doc_id.values[0]
            surface = df_temp[df_temp.ref_id==rid].surface.values[0]
            break
    
    # get all matches
    list_result = []
    for i in range(13):
        df_temp = pd.read_pickle("./pickle/hashsplit/hashsplit{}.pickle".format(i+1))
        list_result.append(df_temp.query('hashtag in @hashes')[['ref_id','doc_id','surface','hashtag']])
    pd_result = pd.concat(list_result)
    
    # with hashes
    pd_result['seed_ref_id'] = rid
    pd_result['seed_doc_id'] = doc_id
    pd_result['seed_doc'] = get_repr(doc_id)
    pd_result['seed_surface'] = surface
    pd_result['doc'] = pd_result.doc_id.apply(get_repr)
    cols = ['seed_ref_id','seed_doc_id','seed_doc','seed_surface','ref_id','doc_id','doc','surface','hashtag']
    pd_result = pd_result[cols]
    
    # without hashes only matches
    pd_csv = pd_result[cols[:-1]]
    pd_csv = pd_csv.drop_duplicates()
    pd_csv['True'] = 0
    pd_csv['Hash'] = 1
    
    length = len(pd_csv)
    
    pd_result.to_csv(f'./sample_csv/{length}_{rid}_hashes.csv')
    pd_csv.to_csv(f'./sample_csv/{length}_{rid}.csv')
    
    return pd_result, pd_csv, length

In [5]:
def find_match(rid):
    print(Reference.objects(_id=rid).first().reference_string)
    pd_result, pd_csv, length = hash_match_csv(rid)
    print(f'Total matches: {length}')

In [10]:
rids = [ObjectId("58e4ce00fe7683152b566af1"),
        ObjectId("58e4ce00fe7683152b55e336"),
        ObjectId("58e4ce00fe7683152b55e33f"),
        ObjectId("58e4ce00fe7683152b55e34f"),
        ObjectId("58e4ce00fe7683152b55e370"),
        ObjectId("58e4ce00fe7683152b55e38b"),
        ObjectId("58e4ce00fe7683152b55e39d"),
        ObjectId("58e4ce00fe7683152b55e39e"),
        ObjectId("58e4ce00fe7683152b55e3a9"),
        ObjectId("58e4ce00fe7683152b560601")]

In [11]:
for i, rid in enumerate(rids): 
    print(i)
    find_match(rid)
    print('\n')

0
Da Canal Vincenzo, Vita di Gregorio Lazzarini [pubblicata da Gian- nantonio Moschini], Venezia, Stamperia Palese, 1809.
Total matches: 180


1
Monticolo, I capitolari delle arti veneziane, voi. n, Roma 1905, p. 467
Total matches: 418


2
Morassi, Novità e precisazioni sul Tiepolo in « Le Arti », 1942, p. 91.
Total matches: 18


3
M. Sañudo, Itinerario per la terra di Venezia nel 1483, Padova, 1847.
Total matches: 118


4
Achille Bosisio, Giosuè Carducci e un letterale Veneziano, in «Ateneo Veneto », Aprile-Giugno 1942, p. 105.
Total matches: 2899


5
Cicogna, Delle inscrizioni veneziane, Venezia 1824-1853, voi. iv, p. 181.
Total matches: 1140


6
G. Pavanello, La Scuola di S. Fan · tin ora Ateneo Veneto, Venezia, 1914;
Total matches: 2886


7
R. Predelli, Le memorie e le carte di Alessandro Vittoria, Trento, 1908.
Total matches: 546


8
Thieme Becker, Künstler Lexikon», Lipsia, 1908, vol. II, pp. 49*50)
Total matches: 94


9
F.X. KRAUS, F. Petrarca e la sua corrispondenza epistolare,

In [12]:
rids_2 = [ObjectId("58e4ce00fe7683152b55e545"),
          ObjectId("58e4ce01fe7683152b58bd7d"),
          ObjectId("58e4ce01fe7683152b58ced4"),
          ObjectId("58e4ce00fe7683152b55e8d0"),
          ObjectId("58e4ce00fe7683152b55e998"),
          ObjectId("58e4ce00fe7683152b55eb67"),
          ObjectId("58e4ce00fe7683152b55ee21"),
          ObjectId("58e4ce00fe7683152b55fd7b"),
          ObjectId("58e4ce00fe7683152b561053"),
          ObjectId("58e4ce00fe7683152b56268f")]

In [13]:
for i, rid in enumerate(rids_2): 
    print(i)
    find_match(rid)
    print('\n')

0
Vita di Apostolo Zeno scrisse Francesco Negri (Venezia, Alvisopoli, 1816),
Total matches: 296


1
Zanipolo catalogue by D. M. Berardelli, published in Nuova Raccolta d’Opuscoli Scientifici e Filologici (ed. Calo- gerà), 32-40 (1778-84),
Total matches: 312


2
Diverse Orationi volgarmente scritte, raccolte per Francesco Sansovino, Venice 1561, ff. 51-54v, especially f. 52 v.
Total matches: 128


3
G. CASANOVA, Istoria delle turbolenze della Polonia , Marsilio, Padova 1974, p. 55.
Total matches: 32


4
Grillo, Venezia. Le difese a mare. Profilo architettonico delle opere di dife- sa idraulica nei litorali di Venezia, Venezia 1989, fig. ΧΧΧΠΙ p. 212.
Total matches: 1870


5
GABRIELLA FERRI PlCCALUGA, Il confine del Nord , Boario Terme 1989, pp. 305-334
Total matches: 56


6
A. Rizzi, Canaletto w Warszawie, p. 68, fig. 20, p. 66;
Total matches: 2


7
Vollard, En écoutant Cézanne, Degas, Renoir, Paris 1838, pp. 201-202.
Total matches: 1


8
FEDALTO, Hierarchia latina, Cala- monensis. 10CO

In [6]:
rid = ObjectId("58e4ce00fe7683152b5639a3")
find_match(rid)

S. Marco, Lettera inedita di Giustina Renier Michiel alla figlia, pubbl. dal dott. Cesare Musatti per le nozze Musatti-Coen, Venezia 1893.
Total matches: 203


In [7]:
rid = ObjectId("58e4ce00fe7683152b5639bd")
find_match(rid)

A. Quatremère de Quincy, Canova et ses ouvrages, Paris 1834, p. 288s.
Total matches: 30


In [8]:
rid = ObjectId("58e4ce00fe7683152b563b99")
find_match(rid)

M. L. Gonelli, La scuola storica, in Storia della letteratura italiana, diretta da
Total matches: 2278


In [9]:
rid = ObjectId("58e4ce00fe7683152b563b9c")
find_match(rid)

M. Moretti, Gentile, D'Ancona e la 'scuola' pisana, «Giornale critico della filosofìa italiana», LXXVIII, 1999, pp. 65-116.
Total matches: 102


In [10]:
rid = ObjectId("58e4ce00fe7683152b563bb3")
find_match(rid)

D. COMPAGNI, La Cronica, introduzione e com- mento di G. Luzzatio, Milano 1906, pp. V- XXXIX, p. XVI.
Total matches: 1


In [11]:
rid = ObjectId("58e4ce01fe7683152b59d6dc")
find_match(rid)

E. Concina, Città e fartele nelle “ tre isole nostre", in Venera e la difesa del Le- vante. Da Lepanto a Candía tyyo-tóyo, Venezia, Arsenale, 1986, p. 189,
Total matches: 431


In [13]:
rid = ObjectId("58e4ce01fe7683152b5a11e8")
find_match(rid)

G. Van Der Sman, L’Eolia di Villa Trento: arte e umanesimo letterario vicentino, in «Arte Veneta», XLII (1988), pp. 58-67.
Total matches: 4037


In [14]:
rid = ObjectId("5919e9663dde944752f5c95b")
find_match(rid)

G. Cracco, Il pensiero storico di fronte ai problemi del comune veneziano , pp. 46-50.
Total matches: 186


In [15]:
rid = ObjectId("5919e9663dde944752f5c95e")
find_match(rid)

A. Limentani, Cinque note su Martino da Canal,  in «Atti dell’Istituto Veneto di Scienze, Lettere ed Arti», 124 (1965-1966),  pp. 257-281;
Total matches: 6394


In [17]:
find_match(ObjectId("5919e9663dde944752f5f058"))

J. Monfasani, George of Trebizond, p. 23 n. 102.
Total matches: 79


In [18]:
find_match(ObjectId("5919e9663dde944752f6790e"))

Corner, Notizie storiche, pp.239-241;
Total matches: 1251


In [19]:
find_match(ObjectId("5919e9663dde944752f67922"))

D. Gutierrez, Testi e note su Mariano da Gennazzano (+1498), «Analecta Augustiniana», XXXII, 1969, pp. 117-204;
Total matches: 9


In [20]:
find_match(ObjectId("59e0adbbf98262071bfa4fb4"))

H. Honour, Chinoiserie. The Vision of Cathay, Londres, 1973, p. 119 sq.
Total matches: 56


In [21]:
find_match(ObjectId("59de7a3348e232540c5f1454"))

Stellini, Poesie originali e tradotte, Padova 1782, pp. 99-245.
Total matches: 18


In [22]:
find_match(ObjectId("59de7a3348e232540c5f5fe7"))

D. Malipiero, Annali veneti dall’anno 145 7 al 1500, Francesco Longo, Firenze 1843, p. 41.
Total matches: 737


In [23]:
find_match(ObjectId("59de7a3348e232540c5f600f"))

M. Brunetti, E. Vitale (a cura di), Corrispondenza da Madrid di Leonardo Dona, Venezia-Roma 1963, t. II, p. 688,
Total matches: 2960


In [24]:
find_match(ObjectId("59df53cf9be3c75c2e686616"))

C. Ghisalberti, La codificazione del diritto in Italia 1865/1942, Bari-Roma 1985, p. 154 ss.;
Total matches: 71


In [25]:
find_match(ObjectId("59df6ec59be3c75dd98bf26b"))

Samonà, Il piano Panfani in rapporto all’attività edilizia dei liberi professionisti, in «Metron», n. 33-34.
Total matches: 153


In [26]:
find_match(ObjectId("58e4ce01fe7683152b59acb8"))

Sapori, Una compagnia di Calimala ai primi del Trecento, Firenze 1932, p. 232.
Total matches: 145


In [27]:
find_match(ObjectId("58e4ce00fe7683152b5625bf"))

TRAMONTIN, A. NIERO, G. MUSOLINO, C. CANDI ANI, Culto dei Santi aVenezia, voi. 2°, Venezia 1965, p. 159- 9
Total matches: 818


In [11]:
rid = ObjectId("58e4ce00fe7683152b55e336")
print(Reference.objects(_id=rid).first().reference_string)
pd_result, pd_csv = hash_match_csv(rid)

Monticolo, I capitolari delle arti veneziane, voi. n, Roma 1905, p. 467


In [8]:
rid = ObjectId("58e4ce00fe7683152b55e33f")
print(Reference.objects(id=rid).first().reference_string)
pd_result, pd_csv = hash_match_csv(rid)

Morassi, Novità e precisazioni sul Tiepolo in « Le Arti », 1942, p. 91.


In [10]:
rid = ObjectId("58e4ce00fe7683152b55e34f")
print(Reference.objects(id=rid).first().reference_string)
pd_result, pd_csv = hash_match_csv(rid)

M. Sañudo, Itinerario per la terra di Venezia nel 1483, Padova, 1847.


In [13]:
rid = ObjectId("58e4ce00fe7683152b55e370")
print(Reference.objects(id=rid).first().reference_string)
pd_result, pd_csv = hash_match_csv(rid)

Achille Bosisio, Giosuè Carducci e un letterale Veneziano, in «Ateneo Veneto », Aprile-Giugno 1942, p. 105.


In [14]:
rid = ObjectId("58e4ce00fe7683152b55e38b")
print(Reference.objects(id=rid).first().reference_string)
pd_result, pd_csv = hash_match_csv(rid)

Cicogna, Delle inscrizioni veneziane, Venezia 1824-1853, voi. iv, p. 181.


In [58]:
rid = ObjectId("58e4ce00fe7683152b55e39d")
print(Reference.objects(id=rid).first().reference_string)
pd_result, pd_csv = hash_match_csv(rid)

G. Pavanello, La Scuola di S. Fan · tin ora Ateneo Veneto, Venezia, 1914;


In [16]:
rid = ObjectId("58e4ce00fe7683152b55e39e")
print(Reference.objects(id=rid).first().reference_string)
pd_result, pd_csv = hash_match_csv(rid)

R. Predelli, Le memorie e le carte di Alessandro Vittoria, Trento, 1908.


In [17]:
rid = ObjectId("58e4ce00fe7683152b55e3a9")
print(Reference.objects(id=rid).first().reference_string)
pd_result, pd_csv = hash_match_csv(rid)

Thieme Becker, Künstler Lexikon», Lipsia, 1908, vol. II, pp. 49*50)


In [18]:
rid = ObjectId("58e4ce00fe7683152b560601")
print(Reference.objects(id=rid).first().reference_string)
pd_result, pd_csv = hash_match_csv(rid)

F.X. KRAUS, F. Petrarca e la sua corrispondenza epistolare, trad. D. VALBUSA; in Firenze, 1901
